In [10]:
import tableauserverclient as tsc
import pandas as pd

In [11]:
user = "blankartz@amendllc.com"
password = "<pass>"
site_id = "statewidesafetysystems"

In [12]:
#tableau_auth = tsc.TableauAuth(user,password,site_id=site_id)


In [13]:
# Ben
#token_name = "tableau-refresh-test"
#token_value = "74l5rXGSTfCt8+Ui/HacXQ==:uxxROtbQvKx3kWbiP6oqvWfmFb1jyfW7"

#andrea
token_name = "refresh-token-ag"
token_value = "53MOfbqBRxSxCgMCrYWukw==:YE5j6jKyT2l3aI5U7Ud8D4VC32b2Ef2f"

In [14]:
tableau_auth = tsc.PersonalAccessTokenAuth(token_name,token_value,site_id=site_id)
tableau_auth

<PersonalAccessToken name=refresh-token-ag token=53MOfbqBRxSxCgMCrYWukw==:YE5j6jKyT2l3aI5U7Ud8D4VC32b2Ef2f>

In [15]:
server = tsc.Server('https://10ay.online.tableau.com/', use_server_version=True)
server

In [16]:
req_option = tsc.RequestOptions()
req_option.filter.add(tsc.Filter(tsc.RequestOptions.Field.ProjectName,
                                 tsc.RequestOptions.Operator.Equals,
                                 "Financial Statements - Sandbox"))

In [17]:
with server.auth.sign_in(tableau_auth):  
    all_workbooks_items, pagination_item = server.workbooks.get(req_option)
    # print names of first 100 workbooks
    for workbook in all_workbooks_items:
        print(workbook)
        print(workbook.name, workbook.id)
        print(workbook.owner_id)


Balance Sheet - Sandbox 8aaf2260-b08e-4f98-8929-74ebf0f3577a
5513ac2f-b6c2-4248-a71e-f08d78d7c76e
Income Statement - Sandbox 99729f01-2df6-484f-921f-ab3fdb7e12ee
0142af8f-a27f-44c5-9a62-ee2c986fed78
Statement of Cash Flow - Sandbox 8125f716-eeb6-49c6-8243-de57725e2ad3
c709306d-49a2-4c14-b932-0b0ca2a0750d
Financial Reporting Analytics - Sandbox bf8b072b-7373-4ad6-b656-57d9c1098f52
0142af8f-a27f-44c5-9a62-ee2c986fed78
Branch Comparison - Sandbox 3ecba336-5fd6-4316-a117-923de0244694
0142af8f-a27f-44c5-9a62-ee2c986fed78
Controllable Cost KPI's - sandbox cc713c90-fc4f-400f-80b0-510a22c31754
0142af8f-a27f-44c5-9a62-ee2c986fed78
Testing Workbook - SoCF Sandbox c4fb9373-84ee-4125-a9df-4d9c428b8c3c
c709306d-49a2-4c14-b932-0b0ca2a0750d


In [18]:
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get()
    print("\nThere are {} user on site: ".format(pagination_item.total_available))
    print([user.name for user in all_users])

ServerResponseError: 

	403000: Forbidden
		'gergel@amendllc.com' isn't authorized to query users on site 'bb9faedf-b881-4f80-becf-43a7e44a2741'.

In [ ]:
job_id = ""
with server.auth.sign_in(tableau_auth):
    # get the workbook item from the site
    workbook = server.workbooks.get_by_id('c4fb9373-84ee-4125-a9df-4d9c428b8c3c')    
    # call the update method
    job = server.workbooks.refresh(workbook)
    print(workbook)
    print(job)
    job_id = job.id
    #print("\nThe data of workbook {0} is refreshed.".format(workbook.name))

<Job#f08438b6-a689-49e7-8304-bd36753ed573 RefreshExtract created_at(2022-04-27 16:19:13+00:00) started_at(None) completed_at(None) progress (None) finish_code(-1)>


Check the status of the job.
https://help.tableau.com/current/api/rest_api/en-us/REST/rest_api_ref.htm#query_job
https://tableau.github.io/server-client-python/docs/api-ref#jobs

finish_code = 1 is a failed job
finish_code = 0 is success 
finish_code = 2 is cancelled
finish_code = -1 is scheduled?  (not listed)

In [ ]:

with server.auth.sign_in(tableau_auth):

     # get the id of the job from response to extract refresh task,
     # or another asynchronous REST API call.
     # in this case, "576b616d-341a-4539-b32c-1ed0eb9db548"


    #myJobId = '874cfafe-2edd-48c5-a70f-97fe8c610ad7'
    #jobinfo = server.jobs.get_by_id(my_job_id)

    my_job_id = job_id
    jobinfo = server.jobs.wait_for_job(my_job_id,timeout=600)
    

    print(jobinfo)
    print(jobinfo.id)
    print(jobinfo.finish_code)
    print(jobinfo.notes)

<Job#f08438b6-a689-49e7-8304-bd36753ed573 RefreshExtract created_at(2022-04-27 16:19:13+00:00) started_at(2022-04-27 16:19:17+00:00) completed_at(2022-04-27 16:20:09+00:00) progress (100) finish_code(0)>
f08438b6-a689-49e7-8304-bd36753ed573
0
["Finished refresh of extracts (new extract id:[hyper:73b4f532-3e5f-4c75-b640-432ca8bd52e8]) for Workbook 'Testing Workbook - SoCF Sandbox'"]


In [ ]:
#server.auth.sign_in(tableau_auth)